<a href="https://colab.research.google.com/github/AlecTraas/computational-geo-lab/blob/main/Colab/Ridge/RangeSearchingV2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
class node:

  def __init__(self, value):
    self.value = value
    self.lchild = None
    self.rchild = None
    self.subtree = None
    self.span = None

  def get_value(self):
    return self.value

  def set_rchild(self, child):
    self.rchild = child

  def set_lchild(self, child):
    self.lchild = child

  def get_rchild(self):
    return self.rchild

  def get_lchild(self):
    return self.lchild

  def is_leaf(self):
    return self.lchild is None and self.rchild is None

  def has_lchild(self):
    return not self.lchild is None

  def has_rchild(self):
    return not self.rchild is None

  def set_subtree(self, root):
    self.subtree = root

  def get_subtree(self):
    return self.subtree

  def set_span(self, nodes):
    self.span = nodes

  def get_span(self):
    return self.span

  def display(self):
    format = ""
    queue = [(self,0)]
    prev_depth = 0
    while len(queue) != 0:
      cur = queue.pop(0)
      curnode = cur[0]
      cur_depth = cur[1]
      if(cur_depth > prev_depth):
        print(format)
        format = ""
        #format += "/n
        prev_depth = cur_depth
      format += str(curnode.value)
      if(curnode.has_lchild()):
        queue.append((curnode.lchild,cur_depth+1))
      if(curnode.has_rchild()):
        queue.append((curnode.rchild,cur_depth+1))

    print(format)

  def compareTo(self, otherNode, nvars, i):
    for j in range(0, nvars): #occurs nvars times
      out = self.value[i] - otherNode.value[i]
      if out !=0:
        return out
      i = (i+1) % nvars
    return 0 #only return 0 if all are same



In [3]:
class database:

  def __init__(self, nvars, values=None):
    self.cur_id = 1
    self.table = []
    if values == None:
      self.size = 0
    else:
      self.size = len(values)
      for i in values:
        self.table += [[self.cur_id]+i]
        self.cur_id += 1

    self.nvars = nvars
    self.id = self.size + 1
    self.db_rt = range_tree(self.table, self.nvars) #check if nvars matches up


  def add(self, value):
    self.table += [[self.cur_id]+value]
    self.size += 1
    self.cur_id += 1


  def update_rt(self):
    self.db_rt = range_tree(self.table, self.nvars) #check if nvars matches up


  def display(self):
    header = "id"
    for i in range (1, self.nvars + 1):
      header += "\tvar" + str(i)
    print(header)
    for i in range(0,self.size):
      print("--------------------")
      point = self.table[i]
      line = ""
      for j in range(0, self.nvars + 1):
        line += str(point[j]) + "\t"
      print(line)

  def query(self, params):
    myquery = range_search_query(self.db_rt.root, params, self.nvars)
    return myquery.answer




In [4]:
class range_tree:

  def __init__(self,points,nvars):
    temp = nvars - 1
    self.root = self.rec_range_tree(points, temp, 0)


  def rec_range_tree(self, points, nvars, i):

    if len(points) == 0:
      return None

    if len(points) == 1:
      root = node(points[0])
      if i < nvars:
        root.set_subtree(node(points[0]))
      else:
        root.set_span(points)

    else:
      median = len(points) // 2
      root = node(points[median])
      root.set_lchild(self.rec_range_tree(points[:median],nvars, i))
      root.set_rchild(self.rec_range_tree(points[median:],nvars, i)) # median included in right tree

      if i < nvars:
        ypoints = sorted(points, key=lambda x: x[i+1]) # sorted on next variable
        root.set_subtree(self.rec_range_tree(ypoints, nvars, i+1))
      else:
        root.set_span(points)

    return root

  def display(self):
    format = ""
    queue = [(self.root,0)]
    prev_depth = 0
    while len(queue) != 0:
      cur = queue.pop(0)
      curnode = cur[0]
      cur_depth = cur[1]
      if(cur_depth > prev_depth):
        print(format)
        format = ""
        #format += "/n
        prev_depth = cur_depth
      format += str(curnode.value)
      if(curnode.has_lchild()):
        queue.append((curnode.lchild,cur_depth+1))
      if(curnode.has_rchild()):
        queue.append((curnode.rchild,cur_depth+1))

    print(format)


In [5]:
class range_search_query:

  def __init__(self, root, params, nvars):
    self.answer = []
    self.rec_query(root, params, nvars, 0)

  def find_split_node(self, root, start, end, i):
    curnode = root
    curval = curnode.get_value()[i]
    while not curnode.is_leaf() and (end < curval or start >= curval):
      if end < curval:
        curnode = curnode.get_lchild()
      else:
        curnode = curnode.get_rchild()
      curval = curnode.get_value()[i]
    return curnode


  def check_leaf(self, leaf, start, end, i):
    value = leaf.get_value()[i]
    return value >= start and value <= end


  def rec_query(self, root, param, nvars, i):

    last_var = i >= nvars
    start = param[i][0]
    end = param[i][1]
    split = self.find_split_node(root, start, end, 0)

    # split is leaf -> check if included
    if split.is_leaf():
      include_leaf = self.check_leaf(split, start, end, i)
      if include_leaf and last_var:
        self.answer += [split.get_value()]
      elif include_leaf:
        self.rec_query(split, param, nvars, i+1)
      return

    # follow left path
    curnode = split.get_lchild()

    while not curnode.is_leaf():
      follow_left = start < curnode.get_value()[i]
      if follow_left and last_var:
        self.answer += [curnode.get_rchild().get_span()]
        curnode = curnode.get_lchild()
      elif follow_left:
        self.rec_query(curnode.get_rchild().get_subtree(), param, nvars, i+1)
        curnode = curnode.get_lchild()
      else:
        curnode = curnode.get_rchild()

    # is terminal leaf included
    if self.check_leaf(curnode, start, end, i):
      if last_var:
        self.answer += [curnode.get_value()]
      else:
        self.rec_query(curnode, param, nvars, i+1)

    # follow right path
    curnode = split.get_rchild()

    while not curnode.is_leaf():
      follow_right = end >= curnode.get_value()[i]
      if follow_right and last_var:
        self.answer += [curnode.get_lchild().get_span()]
        curnode = curnode.get_rchild()
      elif follow_right:
        self.rec_query(curnode.get_lchild().get_subtree(), param, nvars, i+1)
        curnode = curnode.get_rchild()
      else:
        curnode = curnode.get_lchild()

    # is terminal leaf included
    if self.check_leaf(curnode, start, end, i):
      if last_var:
        self.answer += [curnode.get_value()]
      else:
        self.rec_query(curnode, param,nvars,i+1)

In [28]:
mypoints = [[9],[2],[8],[6],[7]]
mypoints = sorted(mypoints, key=lambda x: x[0])
rt = range_tree(mypoints, 1)
rquery= range_search_query(rt.root, [[0,5]], 0)
print(rquery.answer)

[[2]]


In [29]:
mypoints = [[9],[2],[8],[6],[7]]
mypoints = sorted(mypoints, key=lambda x: x[0])
rt = range_tree(mypoints, 1)
rquery= range_search_query(rt.root, [[0,6]], 0)
print(rquery.answer)

[[2], [6]]


In [26]:
mypoints = [[9,20],[2,21],[8,26],[6,25],[7,23]]
mypoints = sorted(mypoints, key=lambda x: x[0])
rt = range_tree(mypoints, 2)
rquery= range_search_query(rt.root, [[0,10],[20,21]], 1)
print(rquery.answer)

[[2, 21], [9, 20]]


In [27]:
mypoints = [[9,20],[2,21],[8,26],[6,25],[7,23]]
mypoints = sorted(mypoints, key=lambda x: x[0])
rt = range_tree(mypoints, 2)
rquery= range_search_query(rt.root, [[0,2],[20,25]], 1)
print(rquery.answer)

[[2, 21]]
